In [1]:
import pandas as pd
import numpy as np

data_file = "~/Documents/ogle/ogle4/smc/RRab.csv"
data = pd.read_csv(data_file).dropna()

del data["id"]
data.describe()

,magnitude_Iband,magnitude_Vband,period,period_uncertainty,time_max_bright,amplitude_Iband,R21,phi21,R31,phi31
count,4748.000000,4748.000000,4748.000000,4.748000e+03,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000
mean,18.839039,19.415382,0.596560,2.803875e-06,6000.297409,0.504458,0.419089,4.294047,0.297968,2.460209
std,0.906234,0.915491,0.057343,3.421723e-06,0.175088,0.153931,0.060421,0.189534,0.070244,0.423121
min,12.048000,12.632000,0.374290,1.000000e-07,6000.000020,0.025000,0.125000,3.206000,0.024000,0.395000
25%,18.918000,19.489000,0.561476,1.200000e-06,6000.148912,0.393000,0.382000,4.163750,0.255000,2.158000
50%,19.062000,19.637000,0.596877,2.000000e-06,6000.294415,0.497000,0.421000,4.266000,0.310000,2.403000
75%,19.182250,19.760000,0.630260,3.300000e-06,6000.442110,0.612000,0.458000,4.400000,0.351000,2.714000
max,20.419000,21.303000,0.975487,1.065000e-04,6000.920100,1.053000,0.654000,5.161000,0.589000,5.795000


In [2]:
from sklearn import linear_model
from sklearn import pipeline
from sklearn import preprocessing

In [3]:
from sklearn.model_selection import train_test_split

# Partition the training and test data
full_x = data.as_matrix(["period", "amplitude_Iband"])
full_y = data.as_matrix(["phi31"])

X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size = 0.20, random_state = 42)

train = pd.DataFrame(np.hstack([X_train, y_train]), columns = ["period", "amplitude_Iband", "phi31"])

train_norm = pd.DataFrame(
    preprocessing.scale(train)
    , columns = ["period", "amplitude_Iband", "phi31"])

In [4]:
from sklearn import svm

# Train the SVM
svm_novelty = svm.OneClassSVM()
svm_novelty.fit(train_norm.as_matrix())

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [5]:
# Find the novelty and regular data points
train["svm_pred"] = svm_novelty.predict(train_norm)

regular_svm = train[train["svm_pred"] == 1.0]
novelty_svm = train[train["svm_pred"] == -1.0]

In [6]:
# Setup for the polynomial regression
d_svm = 2
poly_svm = preprocessing.PolynomialFeatures(degree = d_svm, include_bias = False)
reg_svm = linear_model.LinearRegression()
model_svm = pipeline.Pipeline([("polynomial_features", poly_svm), ("linear_regression", reg_svm)])

# Perform the polynomial regression using cross validation
from sklearn.model_selection import cross_val_score

x_svm = regular_svm.as_matrix(["period", "amplitude_Iband"])
y_svm = regular_svm.as_matrix(["phi31"])

scores = cross_val_score(model_svm, X_train, y_train, cv=5)
print("Cross Validation\n" + "Mean: " + str(scores.mean()) + "\nStd:  " + str(scores.std()))

model_svm.fit(x_svm, y_svm)

Cross Validation
Mean: 0.441432847608
Std:  0.0432585208297


Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('linear_regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [7]:
print("Test Score: " + str(model_svm.score(X_test, y_test)))

Test Score: 0.451286269254
